In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
with open(filter_list_company_name_path) as f:
    filter_list_company_name = f.read().splitlines()
with open(filter_list_company_name_part_path) as f:
    filter_list_company_name_part = f.read().splitlines()

with open(filter_list_posting_region_path) as f:
    filter_list_posting_region = f.read().splitlines()
with open(filter_list_posting_name_part_path) as f:
    filter_list_posting_name_part = f.read().splitlines()

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5935


### 연결 확인 (200, 400)

In [2]:
import requests
search_link = 'https://www.naver.com/'
response = requests.get(search_link)
print(response)

<Response [200]>


### JSON 파일 불러오기

In [4]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5935


### 회사 채용공고 리스트 확인

In [9]:
company_name = "리콘랩스"
for i in job_group[company_name]:
    print(i['title'])
print("---------------------")
print(len(job_group[company_name]))

### 회사명 부분적 단어 포함 (전체현황)

In [5]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                    nCnt += 1

    print(filtering_word, nCnt)

학원 177
스터디 10
교육원 18
교육센터 7
평생교육원 3
필라테스 5
아카데미 194
병원 38
부동산 36
중개 45
건축사 7
의원 42
결혼 12
출판 8
손해사정 5
축구 2
스쿨 16
학교 64
산부인과 2
주얼리 5
부티크 1
복지관 2
스투디오 1
에스씨에이게임 34
렌트카 1
스타트업 4
의료센터 3
웨딩 6
에이치알 65
테라로사 7
성형외과 18
변호사 2


### 회사명 부분적 단어 포함 (하나씩 확인)

In [6]:
filtering_word = "변호사"
ans_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['company']:
            ans_list.append(company_title)
print(filtering_word)
print("검출개수: " + str(len(ans_list)))
ans_list = list(set(ans_list))
print("중복제거: " + str(len(ans_list)))
print("----------------------")
for ans in ans_list:
    print(ans)

변호사
검출개수: 2
중복제거: 1
----------------------
변호사이원후법률사무소


### 공고명 부분적 단어 포함 (전체현황)

In [21]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 49
매장관리 13
고객 171
사무보조 100
마케팅 1035
마케터 399
경영지원 277
NodeJs 1
Nestjs 1
부동산 75
영업사원 18
자재관리 10
회계 196
재무 82
총무 121
영상의학실 1
유지보수 125
고객응대 4
텔레마케터 1
아웃바운드 11
국비지원 7
화장품 130
디자이너 601
인플루언서 13
MD 413
AMD 91
온라인 510
네일리스트 1
웹디자인 69
기술영업 98
프론트엔드 163
회로설계 11
백엔드 271
강사 199
경리 54
사무행정 10
보험설계사 25
유튜브 47
유투브 1
국비무료 9
국비지원 7
수행기사 2
행정보조 2
구축사업 14
골프장 1
영업지원 77
Node.js 12
node.js 3
방송장비 3
물류팀 14
물류관리 17
창고관리 4
정보보안 32
품질경영 7
코스메틱 31
자동차튜닝 3
시각디자인 10
악세사리 2
업무제휴 3
무료 28
영상제작 13
생산관리 31
인사팀장 2
자동차영업 2
생산팀 11
창업센터 1
가맹점 5


### 공고명 부분적 단어 포함 (하나씩 확인)

In [20]:
filtering_word = "백엔드"
company_list = []
posting_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['title']:
            company_list.append(company_title)
            posting_list.append(job_list['title'])
print(filtering_word)
print("검출개수: " + str(len(company_list)))
print("----------------------")
for idx in range(len(company_list)):
    print(company_list[idx] + " || " + posting_list[idx])

백엔드
검출개수: 271
----------------------
큐픽스 || 백엔드 개발
큐픽스 || [신입/경력] 백엔드, AWS/ELK/Terraform (병역특례 가능)
큐픽스 || [신입/경력] 백엔드, AWS/ELK/Terraform, 병역특례 가능
라이트비전 || (경력)백엔드 개발자 채용
리콘랩스 || [리콘랩스] 백엔드 개발자 및 AI 엔지니어 모집
리콘랩스 || [리콘랩스] 테크 부문(백엔드/AI 엔지니어) 신입 및 경력 모집
한국축산데이터 || 서비스기획자, PO, 백엔드개발자, Data Analyst 모집 (경력)
라온피플 || AI 서비스 플랫폼 백엔드 개발
코비젼 || 개발직 경력직 채용-JAVA/.NET/PM/백엔드/프론트엔드/인프라/웹기획
카카오 || [get Server!] 비즈플랫폼 백엔드 경력 개발자 영입
핫셀러 || 글로벌 서비스를 만들어 갈 Python(파이썬) 백엔드 개발자 경력직 채용
핫셀러 || 글로벌 서비스를 만들어 갈 Python(파이썬) 백엔드 개발자 팀장급 채용
와이즈버즈 || 백엔드 개발자 (3년 이상)
트루엔 || 백엔드 개발 (경력)
이글루코퍼레이션 || 서버 백엔드 개발
이글루코퍼레이션 || 웹 어플리케이션 백엔드 개발
이글루코퍼레이션 || 연구소 IT개발(서버/백엔드/프론트엔드/Python/웹 퍼블리셔/QA 엔지니어) 대규모 채용
바질컴퍼니 || 백엔드 개발자(웹 서버/SDK 개발) 채용(신입/경력)
바질컴퍼니 || 개발자(백엔드, 응용S/W) 채용(신입/경력)
메씨인터내셔날 || JAVA 백엔드 웹개발 신입 모집 - 재택가능
와따 || [병역특례전문연구원] WATA Inc 백엔드 개발자 모집
와따 || [WATA Inc] 시니어 백엔드 개발직군 모집
와따 || [WATA Inc] 리더 백엔드 개발자 모집
메이크잇 || 메이크잇(Letsmakeit) 금융핀테크 기업 서버/백엔드 개발자 채용
브리치 || [강소기업/청년내일채움공제]주식회사 브리치 서비스개발 및 기획부분(백엔드&프런트앤드 개발자,머신러닝&

### 지역명 단어 포함 (전체현황)

In [22]:
for filtering_word in filter_list_posting_region:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                    nCnt += 1
                    
    print(filtering_word, nCnt)

대전 57
대구 66
부산 103
인천 74
세종 16
일본 43
김해 7
전북 3
광주 49
제주 14
전북 3
창원 14
천안 14
청주 13
인천 74
충주 2
진주 1
화성 9
동탄 29


### 지역명 단어 포함 (하나씩 확인)

In [40]:
filtering_word = "동탄"
company_list = []
posting_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['title']:
            company_list.append(company_title)
            posting_list.append(job_list['title'])
print(filtering_word)
print("검출개수: " + str(len(company_list)))
print("----------------------")
for idx in range(len(company_list)):
    print(company_list[idx] + " || " + posting_list[idx])

동탄
검출개수: 29
----------------------
피아이이 || [동탄/오산] (주)피아이이 필드 엔지니어 모집
자비스 || 동탄제조센터 구매팀 신입사원(사원) 채용
자비스 || 동탄제조센터 X-RAY 검사장비 운영(S/W)팀 연구원 채용
자비스 || 동탄제조센터 구매팀 신입사원(사원급) 채용
자비스 || 동탄제조센터 구매팀 신입사원(사원급) 모집
자비스 || 동탄제조센터 X-RAY 검사장비 운영(S/W)팀 연구원 모집
우성 || [키츠가든/동탄] 펫푸드 온라인 마케팅MD 경력 채용
스맥 || [현풍/동탄]스맥 PC프로그램 제어 설계 개발자 채용
에이치엘에스 || [화성시/동탄/주5일] 아웃소싱 마케팅직원 모집
에이치엘에스 || [화성시/동탄] 아웃소싱 잡매니저 모집
에이치엘에스 || [화성시/동탄/주5일] 아웃소싱 잡매니저 모집
주식회사새날 || [동탄][경력/신입](주)새날 본사-기획부문(사업기획/제안/문서 작성/컨텐츠기획)사무직 채용
주식회사새날 || [동탄][경력](주)새날 본사-WEB(PHP, 프론트, 백엔드, API, react.js) 개발 / 하이브리드 APP 개발자 채용
박승철헤어스투디오 || 박승철헤어스투디오 동탄홈플러스점 매장 관리자
잇츠미의원 || (월급225이상)잇츠미의원(피부과)동탄점에서 코디네이터 선생님을 모집합니다.
한국정보기술 || 네트워크 유지관리 OP담당 채용[화성시 동탄]
한국정보기술 || 네트워크 유지관리 인력 채용[화성 동탄, 충북 진천, 기타 전국구]
삼성화재해상보험 || 경기 동탄지역 삼성화재 보험설계사 모집(동탄 보험설계사 모집)
미스터아빠 || 동탄점 야채과일 푸드마켓 미스터아빠 주말파트 근무자 모집
메디코 || 동탄 초고속성장기업) 바쿠나 웹디/경리/뷰티매니저 대규모 공개채용
디어젠 || [디어젠] 동탄R&D 신약연구소 약리연구팀 채용
디어젠 || [디어젠] 동탄R&D 신약연구소 연구지원팀 수시 채용
테라로사 || 테라로사 동탄호수점 레스토랑(피제리아) 조리/홀서비스 채용
학산테라로사커피성수지점 || 